### 1.0 fragment_Voc2

In [2]:
import os

# Encode a fragment.
from rdkit import Chem
from fragment_utils.mol_utils import join_fragments
from fragment_utils.mol_utils import split_molecule
import pandas as pd
import random

def encode_molecule(m):
    fs_list = []
    count = 0
    error_smiles = []
    for index,i in enumerate(m):
        try:
            fs = [Chem.MolToSmiles(f) for f in split_molecule(Chem.MolFromSmiles(i))]
            # encoded = "-".join([encodings[f] for f in fs])
            fs_list.extend(fs)
            # print(index)
        except Exception as e:
            error_smiles.append(i)
  
    return fs_list, error_smiles

def decode_molecule(enc):
    fs_list = []
    for index, i in enumerate(enc):
        fs = [Chem.MolFromSmiles(x) for x in i]
        fs = join_fragments(fs)
        fs_list.append(Chem.MolToSmiles(fs))
        print(index)
    return fs_list

def decode_molecule_test(enc):
    fs_list = []
    fs = [Chem.MolFromSmiles(x) for x in enc]
    fs = join_fragments(fs)
    fs_list.append(Chem.MolToSmiles(fs))
    return fs_list


smiles = pd.read_csv('./data/Transformer/01_succeed_smiles.csv',header=None).values.flatten().tolist()
smiles = random.sample(smiles,3500000)
print(len(smiles))

enc,drop_smiles = encode_molecule(smiles)


smiles_filtered = list(set(smiles)-set(drop_smiles))
pd.DataFrame(smiles_filtered).to_csv('./data/Transformer/data_set.csv',header=None,index=None)

with open('./data/Transformer/fragments_counts.txt',"a") as f:
    f.write("the fragment number of data (not drop duplicated):  " + str(len(enc))+'\n')
    f.write('the smiles number of data:  '+ str(len(smiles_filtered))+'\n')
    f.close()


output = list(set(enc))
pd.DataFrame(output).to_csv('./data/fragments_Voc2.csv',header=False,index=False)


# frg_list = ['C[Yb]',  '[Yb]C1CCN([Lu])CC1', '[Yb]N[Lu]', '[Yb]c1cc([Lu])ccc1[Ta]','[Yb]c1cc2c([Lu])ncnc2[nH]1', '[Yb]O[Lu]', '[Yb]c1ccc2oc([Lu])nc2c1', '[Yb]N[Lu]', '[Yb]c1ccc([Lu])cc1', 'Cl[Yb]', 'C[Yb]']

# dec = decode_molecule_test(enc)

# print(dec)

3500000


/home/kara_chen/anaconda3/envs/amp/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
[09:14:37] Explicit valence for atom # 10 Rn greater than permitted
[09:22:38] Explicit valence for atom # 10 Rn greater than permitted
[09:42:03] Explicit valence for atom # 10 Rn greater than permitted
[09:49:31] Explicit valence for atom # 10 Rn greater than permitted


### 2.0 训练集与测试集数据拆分

In [7]:

import pandas as pd
from sklearn.model_selection import train_test_split
import random


# 从CSV文件中加载数据集
data = pd.read_csv('./data/Transformer/data_set.csv').values.flatten().tolist()



def split_list(input_list):
    # 随机打乱列表
    random.shuffle(input_list)

    # 计算每个部分的大小
    n = len(input_list)
    train_size = int(0.8 * n)
    valid_size = int(0.1 * n)
    
    # 切分列表
    train_list = input_list[:train_size]
    valid_list = input_list[train_size:train_size + valid_size]
    test_list = input_list[train_size + valid_size:]

    return train_list, valid_list, test_list


# 拆分列表
train, valid, test = split_list(data)

print("Train List:", len(train))
print("Validation List:", len(valid))
print("Test List:", len(test))

pd.DataFrame(train,columns=['smiles']).to_csv('./data/gen_train.csv')
pd.DataFrame(valid,columns=['smiles']).to_csv('./data/gen_valid.csv')
pd.DataFrame(test,columns=['smiles']).to_csv('./data/gen_test.csv')


Train List: 2799435
Validation List: 349929
Test List: 349930
